# 7. Data Cleaning and Preparation


During the course of doing data analysis and modeling, a significant amount of time
is spent on data preparation: loading, cleaning, transforming, and rearranging. Such
tasks are often reported to take up 80% or more of an analyst’s time. Sometimes the
way that data is stored in files or databases is not in the right format for a particular
task. Many researchers choose to do ad hoc processing of data from one form to
another using a general-purpose programming language, like Python, Perl, R, or Java,
or Unix text-processing tools like sed or awk. Fortunately, pandas, along with the
built-in Python language features, provides you with a high-level, flexible, and fast set
of tools to enable you to manipulate data into the right form.

## 7.1 Handling Missing Data

Missing data occurs commonly in many data analysis applications. One of the goals
of pandas is to make working with missing data as painless as possible. For example,
all of the descriptive statistics on pandas objects exclude missing data by default. <br>
The way that missing data is represented in pandas objects is somewhat imperfect,
but it is functional for a lot of users. For numeric data, pandas uses the floating-point
value NaN (Not a Number) to represent missing data. We call this a sentinel value that
can be easily detected:

Lets start with a simple example: <br>
Importing desired libraries

In [1]:
import pandas as pd
from numpy import nan as NA
import numpy as np

In [2]:
string_data = pd.Series(['aardvark', 'artichoke', np.nan, 'avocado'])
string_data

0     aardvark
1    artichoke
2          NaN
3      avocado
dtype: object

In [3]:
string_data.isnull()

0    False
1    False
2     True
3    False
dtype: bool

In pandas, we’ve adopted a convention used in the R programming language by refer‐
ring to missing data as NA, which stands for not available. In statistics applications,
NA data may either be data that does not exist or that exists but was not observed
(through problems with data collection, for example). When cleaning up data for
analysis, it is often important to do analysis on the missing data itself to identify data
collection problems or potential biases in the data caused by missing data.

The built-in Python None value is also treated as NA in object arrays:


In [4]:
string_data[0] = None
string_data

0         None
1    artichoke
2          NaN
3      avocado
dtype: object

In [5]:
string_data.isnull()

0     True
1    False
2     True
3    False
dtype: bool

### **Filtering Out Missing Data**

There are a few ways to filter out missing data. While you always have the option to
do it by hand using pandas.isnull and boolean indexing, the dropna can be helpful.
On a Series, it returns the Series with only the non-null data and index values:

In [6]:
data = pd.Series([1, NA, 3.5, NA])
data

0    1.0
1    NaN
2    3.5
3    NaN
dtype: float64

In [7]:
data.dropna()

0    1.0
2    3.5
dtype: float64

This is equivalent to:


In [8]:
data[data.notnull()], data[data.notna()], data[~data.isna()], data[~data.isnull()]

(0    1.0
 2    3.5
 dtype: float64,
 0    1.0
 2    3.5
 dtype: float64,
 0    1.0
 2    3.5
 dtype: float64,
 0    1.0
 2    3.5
 dtype: float64)

With DataFrame objects, things are a bit more complex. You may want to drop rows
or columns that are all NA or only those containing any NAs. dropna by default drops
any row containing a missing value:

In [9]:
data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA], [NA, NA, NA], [NA, 6.5, 3.]])
data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [10]:
cleaned = data.dropna()
cleaned

,0,1,2
0,1.0,6.5,3.0


Passing how='all' will only drop rows that are all NA:


In [11]:
data.dropna(how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


In [12]:
data[4] = NA
data

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [13]:
data.dropna(how='all', axis=1)

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


A related way to filter out DataFrame rows tends to concern time series data. Suppose
you want to keep only rows containing a certain number of observations. You can
indicate this with the thresh argument:

In [14]:
df = pd.DataFrame(np.random.randn(7, 3))
df

,0,1,2
0,0.538333,0.201114,1.341932
1,-0.584162,0.916118,0.353318
2,-0.125505,0.577822,0.732900
3,-1.034246,-0.437294,-0.882064
4,0.588905,0.499734,-0.398193
5,-0.333121,-0.784318,0.544111
6,-0.438575,-0.818611,2.449219


In [15]:
df.iloc[:4, 1] = NA

In [16]:
df

,0,1,2
0,0.538333,NaN,1.341932
1,-0.584162,NaN,0.353318
2,-0.125505,NaN,0.732900
3,-1.034246,NaN,-0.882064
4,0.588905,0.499734,-0.398193
5,-0.333121,-0.784318,0.544111
6,-0.438575,-0.818611,2.449219


In [17]:
df.iloc[:2, 2] = NA
df

,0,1,2
0,0.538333,NaN,NaN
1,-0.584162,NaN,NaN
2,-0.125505,NaN,0.732900
3,-1.034246,NaN,-0.882064
4,0.588905,0.499734,-0.398193
5,-0.333121,-0.784318,0.544111
6,-0.438575,-0.818611,2.449219


In [18]:
df.dropna()

,0,1,2
4,0.588905,0.499734,-0.398193
5,-0.333121,-0.784318,0.544111
6,-0.438575,-0.818611,2.449219


In [19]:
df

,0,1,2
0,0.538333,NaN,NaN
1,-0.584162,NaN,NaN
2,-0.125505,NaN,0.732900
3,-1.034246,NaN,-0.882064
4,0.588905,0.499734,-0.398193
5,-0.333121,-0.784318,0.544111
6,-0.438575,-0.818611,2.449219


In [20]:
df.dropna(thresh=2, axis=1)

,0,1,2
0,0.538333,NaN,NaN
1,-0.584162,NaN,NaN
2,-0.125505,NaN,0.732900
3,-1.034246,NaN,-0.882064
4,0.588905,0.499734,-0.398193
5,-0.333121,-0.784318,0.544111
6,-0.438575,-0.818611,2.449219


thresh=N requires that a row/column has at least N non-NaNs to survive

Rather than filtering out missing data (and potentially discarding other data along
with it), you may want to fill in the “holes” in any number of ways. For most pur‐
poses, the fillna method is the workhorse function to use. Calling fillna with a
constant replaces missing values with that value:

In [21]:
df.fillna(0)

,0,1,2
0,0.538333,0.000000,0.000000
1,-0.584162,0.000000,0.000000
2,-0.125505,0.000000,0.732900
3,-1.034246,0.000000,-0.882064
4,0.588905,0.499734,-0.398193
5,-0.333121,-0.784318,0.544111
6,-0.438575,-0.818611,2.449219


Calling fillna with a dict, you can use a different fill value for each column:


In [22]:
df.fillna({1:0.5, 2:1})

,0,1,2
0,0.538333,0.500000,1.000000
1,-0.584162,0.500000,1.000000
2,-0.125505,0.500000,0.732900
3,-1.034246,0.500000,-0.882064
4,0.588905,0.499734,-0.398193
5,-0.333121,-0.784318,0.544111
6,-0.438575,-0.818611,2.449219


fillna returns a new object, but you can modify the existing object in-place:


In [23]:
df.fillna(0, inplace=True)
df

,0,1,2
0,0.538333,0.000000,0.000000
1,-0.584162,0.000000,0.000000
2,-0.125505,0.000000,0.732900
3,-1.034246,0.000000,-0.882064
4,0.588905,0.499734,-0.398193
5,-0.333121,-0.784318,0.544111
6,-0.438575,-0.818611,2.449219


The same interpolation methods available for reindexing can be used with fillna:


In [24]:
df = pd.DataFrame(np.random.randn(7,3))
df

,0,1,2
0,-0.387781,-1.581871,1.068230
1,0.259585,-1.252693,0.565451
2,-1.219558,-0.866461,-0.736952
3,-1.534886,1.773351,-0.978173
4,-0.436855,-3.563979,0.218763
5,1.239772,0.897250,0.529361
6,-1.096113,-0.325967,0.372840


In [25]:
df.iloc[2:, 1] = NA
df

,0,1,2
0,-0.387781,-1.581871,1.068230
1,0.259585,-1.252693,0.565451
2,-1.219558,NaN,-0.736952
3,-1.534886,NaN,-0.978173
4,-0.436855,NaN,0.218763
5,1.239772,NaN,0.529361
6,-1.096113,NaN,0.372840


In [26]:
df.iloc[4:, 2] = NA
df

,0,1,2
0,-0.387781,-1.581871,1.068230
1,0.259585,-1.252693,0.565451
2,-1.219558,NaN,-0.736952
3,-1.534886,NaN,-0.978173
4,-0.436855,NaN,NaN
5,1.239772,NaN,NaN
6,-1.096113,NaN,NaN


In [27]:
df.fillna(method= 'ffill')

,0,1,2
0,-0.387781,-1.581871,1.068230
1,0.259585,-1.252693,0.565451
2,-1.219558,-1.252693,-0.736952
3,-1.534886,-1.252693,-0.978173
4,-0.436855,-1.252693,-0.978173
5,1.239772,-1.252693,-0.978173
6,-1.096113,-1.252693,-0.978173


In [28]:
df.fillna(method='ffill', limit=2)

,0,1,2
0,-0.387781,-1.581871,1.068230
1,0.259585,-1.252693,0.565451
2,-1.219558,-1.252693,-0.736952
3,-1.534886,-1.252693,-0.978173
4,-0.436855,NaN,-0.978173
5,1.239772,NaN,-0.978173
6,-1.096113,NaN,NaN


With fillna you can do lots of other things with a little creativity. For example, you
might pass the mean or median value of a Series:

In [29]:
data = pd.Series([1., NA, 3.5, NA, 7.])
data

0    1.0
1    NaN
2    3.5
3    NaN
4    7.0
dtype: float64

In [30]:
data.fillna(data.mean())

0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64

## 7.2 Data Transformation

### **Removing Duplicates**

Duplicate rows may be found in a DataFrame for any number of reasons. Here is an
example:

In [31]:
['one', 'two']*3

['one', 'two', 'one', 'two', 'one', 'two']

In [32]:
data = pd.DataFrame({'k1':['one', 'two']*3 + ['two'], 'k2':[1,1,2,3,3,4,4]})
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


The DataFrame method duplicated returns a boolean Series indicating whether each
row is a duplicate (has been observed in a previous row) or not:

In [33]:
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

Relatedly, drop_duplicates returns a DataFrame where the duplicated array is
False:

In [34]:
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


Both of these methods by default consider all of the columns; alternatively, you can
specify any subset of them to detect duplicates. Suppose we had an additional column
of values and wanted to filter duplicates only based on the 'k1' column:

In [35]:
data['v1'] = range(7)
data

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


In [36]:
data.duplicated(['k1'], keep='last')

0     True
1     True
2     True
3     True
4    False
5     True
6    False
dtype: bool

In [37]:
data.drop_duplicates(['k1'], keep='last')

,k1,k2,v1
4,one,3,4
6,two,4,6


In [38]:
data

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


In [39]:
data.drop_duplicates(['k1', 'k2'], keep='last')

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


### **Transforming Data Using a Function or Mapping**

In [40]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon',
     'Pastrami', 'corned beef', 'Bacon',
     'pastrami', 'honey ham', 'nova lox'],
     'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


Suppose you wanted to add a column indicating the type of animal that each food
came from. Let’s write down a mapping of each distinct meat type to the kind of
animal:

In [41]:
meat_to_animal = {'bacon': 'pig', 'pulled pork': 'pig', 'pastrami': 'cow', 'corned beef': 'cow',
                        'honey ham': 'pig',
                        'nova lox': 'salmon'}
meat_to_animal

{'bacon': 'pig',
 'pulled pork': 'pig',
 'pastrami': 'cow',
 'corned beef': 'cow',
 'honey ham': 'pig',
 'nova lox': 'salmon'}

In [42]:
lowercased = data['food'].str.lower()
lowercased

0          bacon
1    pulled pork
2          bacon
3       pastrami
4    corned beef
5          bacon
6       pastrami
7      honey ham
8       nova lox
Name: food, dtype: object

In [43]:
data['animal'] = lowercased.map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


We could also have passed a function that does all the work:


In [44]:
meat_to_animal

{'bacon': 'pig',
 'pulled pork': 'pig',
 'pastrami': 'cow',
 'corned beef': 'cow',
 'honey ham': 'pig',
 'nova lox': 'salmon'}

In [45]:
data['food']

0          bacon
1    pulled pork
2          bacon
3       Pastrami
4    corned beef
5          Bacon
6       pastrami
7      honey ham
8       nova lox
Name: food, dtype: object

In [46]:
data['animal'] = data['food'].map(lambda x: meat_to_animal[x.lower()])
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


Using map is a convenient way to perform element-wise transformations and other
data cleaning–related operations.

### **Replacing Values**

Filling in missing data with the fillna method is a special case of more general value
replacement. As you’ve already seen, map can be used to modify a subset of values in
an object but replace provides a simpler and more flexible way to do so. Let’s con‐
sider this Series:

In [47]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

The -999 values might be sentinel values for missing data. To replace these with NA
values that pandas understands, we can use replace, producing a new Series (unless
you pass inplace=True):

In [48]:
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

If you want to replace multiple values at once, you instead pass a list and then the
substitute value:

In [49]:
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

To use a different replacement for each value, pass a list of substitutes:


In [50]:
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

The argument passed can also be a dict:


In [51]:
data.replace({-999:np.nan, -1000:0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

### **Renaming Axis Indexes**

Like values in a Series, axis labels can be similarly transformed by a function or map‐
ping of some form to produce new, differently labeled objects. You can also modify
the axes in-place without creating a new data structure. Here’s a simple example:

In [52]:
data = pd.DataFrame(np.arange(12).reshape(3,4), index=['Ohio', 'Colorado', 'New York'], columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [53]:
data.index.map(lambda x: x[:4].upper())

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

In [54]:
data.index = data.index.map(lambda x: x[:4].upper())
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


If you want to create a transformed version of a dataset without modifying the origi‐
nal, a useful method is rename:

In [55]:
data.rename(index=str.title, columns=str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


Notably, rename can be used in conjunction with a dict-like object providing new val‐
ues for a subset of the axis labels:

In [56]:
data.rename(index={'OHIO':'Indiana'}, columns={'three':'Peakaboo'})

,one,two,Peakaboo,four
Indiana,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


rename saves you from the chore of copying the DataFrame manually and assigning
to its index and columns attributes. Should you wish to modify a dataset in-place,
pass inplace=True:

In [57]:
data.rename(index={'OHIO':'Indiana'}, columns={'three':'Peakaboo'}, inplace=True)
data

,one,two,Peakaboo,four
Indiana,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


### **Discretization and Binning**


Continuous data is often discretized or otherwise separated into “bins” for analysis.
Suppose you have data about a group of people in a study, and you want to group
them into discrete age buckets:

In [58]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]
ages

[20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

Let’s divide these into bins of 18 to 25, 26 to 35, 36 to 60, and finally 61 and older. To
do so, you have to use cut, a function in pandas:

In [59]:
bins = [18, 25, 35, 60, 100]
bins

[18, 25, 35, 60, 100]

In [60]:
cats = pd.cut(ages, bins)
cats

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64, right]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

The object pandas returns is a special Categorical object. The output you see
describes the bins computed by pandas.cut. You can treat it like an array of strings
indicating the bin name; internally it contains a categories array specifying the dis‐
tinct category names along with a labeling for the ages data in the codes attribute:

In [61]:
cats.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [62]:
cats.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]], dtype='interval[int64, right]')

In [63]:
pd.value_counts(cats)

(18, 25]     5
(25, 35]     3
(35, 60]     3
(60, 100]    1
dtype: int64

Note that pd.value_counts(cats) are the bin counts for the result of pandas.cut.


In [64]:
pd.cut(ages, [18, 25, 35, 60, 100], right=False)

[[18, 25), [18, 25), [25, 35), [25, 35), [18, 25), ..., [25, 35), [60, 100), [35, 60), [35, 60), [25, 35)]
Length: 12
Categories (4, interval[int64, left]): [[18, 25) < [25, 35) < [35, 60) < [60, 100)]

You can also pass your own bin names by passing a list or array to the labels option:


In [65]:
pd.cut(ages, bins, labels=['Youth', 'YoungAdult', 'MiddleAged', 'Senior'])

['Youth', 'Youth', 'Youth', 'YoungAdult', 'Youth', ..., 'YoungAdult', 'Senior', 'MiddleAged', 'MiddleAged', 'YoungAdult']
Length: 12
Categories (4, object): ['Youth' < 'YoungAdult' < 'MiddleAged' < 'Senior']

If you pass an integer number of bins to cut instead of explicit bin edges, it will com‐
pute equal-length bins based on the minimum and maximum values in the data.
Consider the case of some uniformly distributed data chopp

In [66]:
data = np.random.rand(20)
data

array([0.08324033, 0.32862319, 0.71911801, 0.27280691, 0.60255643,
       0.63989967, 0.4135779 , 0.95704181, 0.50503121, 0.61311122,
       0.18504804, 0.47021529, 0.90874644, 0.62459442, 0.83921779,
       0.17047296, 0.38556617, 0.90668858, 0.01898855, 0.63364757])

In [67]:
pd.cut(data, 2, precision=2)

[(0.018, 0.49], (0.018, 0.49], (0.49, 0.96], (0.018, 0.49], (0.49, 0.96], ..., (0.018, 0.49], (0.018, 0.49], (0.49, 0.96], (0.018, 0.49], (0.49, 0.96]]
Length: 20
Categories (2, interval[float64, right]): [(0.018, 0.49] < (0.49, 0.96]]

The precision=2 option limits the decimal precision to two digits.


A closely related function, qcut, bins the data based on sample quantiles. Depending
on the distribution of the data, using cut will not usually result in each bin having the
same number of data points. Since qcut uses sample quantiles instead, by definition
you will obtain roughly equal-size bins:

In [68]:
data = np.random.randn(100)
data

array([-0.03787747, -0.13324305, -0.47323472, -0.11944723, -1.12555776,
       -0.61102908,  0.56199188, -1.30677669, -1.77376218,  0.70347006,
        2.51102347,  0.59813426,  0.12713976,  0.91472642, -0.26857523,
        0.91681083,  1.06962936, -0.71158735, -0.44618938, -0.52414975,
        0.45663684,  0.26604849,  0.40350146,  1.08452716, -0.75304476,
       -0.91609506,  0.96433647, -0.63169427, -0.44959017,  1.13035623,
        0.60972419,  0.12035036,  0.18195853, -0.70864368,  0.50227319,
       -0.43201689,  0.2383722 ,  0.09564951, -0.60367386, -0.36895512,
       -0.51659182, -1.10924902,  0.05733286, -1.27189593, -1.22483029,
        0.72042735,  1.71312608, -0.30673727, -0.37265196,  1.29217054,
       -0.32665737, -1.0666138 ,  0.6041076 ,  0.64412073, -0.04231317,
       -1.51771093, -2.54625622, -0.5397548 , -0.62891272, -0.31173736,
       -1.17784627,  0.68072875,  0.21564791,  0.97959639, -0.82334448,
        0.99225106,  0.23366107,  0.61558935,  0.8790032 ,  0.31

In [69]:
cats = pd.qcut(data, 7)
cats

[(-0.417, -0.0347], (-0.417, -0.0347], (-0.599, -0.417], (-0.417, -0.0347], (-2.5469999999999997, -1.094], ..., (0.833, 2.511], (-0.0347, 0.304], (-1.094, -0.599], (-0.417, -0.0347], (-0.417, -0.0347]]
Length: 100
Categories (7, interval[float64, right]): [(-2.5469999999999997, -1.094] < (-1.094, -0.599] < (-0.599, -0.417] < (-0.417, -0.0347] < (-0.0347, 0.304] < (0.304, 0.833] < (0.833, 2.511]]

In [70]:
pd.value_counts(cats)

(-2.5469999999999997, -1.094]    15
(0.833, 2.511]                   15
(-1.094, -0.599]                 14
(-0.599, -0.417]                 14
(-0.417, -0.0347]                14
(-0.0347, 0.304]                 14
(0.304, 0.833]                   14
dtype: int64

### **Detecting and Filtering Outliers**

Filtering or transforming outliers is largely a matter of applying array operations.
Consider a DataFrame with some normally distributed data:

In [71]:
data = pd.DataFrame(np.random.randn(1000, 4))
data

,0,1,2,3
0,-1.254021,-0.831921,0.092133,-0.842859
1,1.749094,-0.321144,-0.494833,1.106043
2,1.019539,0.329996,0.697000,-0.093604
3,-0.743860,1.392993,-0.796922,-1.036154
4,-1.748362,3.105269,2.014777,-1.494419
...,...,...,...,...
995,-0.825264,-1.334238,-0.726399,1.311375
996,-0.121421,-0.058222,-0.363359,-0.728388
997,0.410431,0.001829,0.735188,-0.946308
998,-1.467808,0.817634,-1.005668,-1.845859


In [72]:
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.028309,0.039326,0.022261,0.024670
std,1.029333,1.017606,0.987705,1.004988
min,-3.222834,-2.663346,-2.780534,-3.102184
25%,-0.744103,-0.634721,-0.655190,-0.639902
50%,-0.032687,-0.011789,0.015264,0.041739
75%,0.668105,0.756746,0.714148,0.697417
max,3.532026,3.105269,2.897205,3.603309


In [73]:
data[data[2].apply(lambda x: x>3)]

,0,1,2,3


In [74]:
(data[data[2] > 3])

,0,1,2,3


In [75]:
col = data[2]

In [76]:
col[np.abs(col)>3]

Series([], Name: 2, dtype: float64)

In [77]:
data[(np.abs(data) > 3).any(1)]

C:\Users\muhammad.adil\AppData\Local\Temp\ipykernel_23704\2969116195.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.any and Series.any will be keyword-only.
  data[(np.abs(data) > 3).any(1)]


,0,1,2,3
4,-1.748362,3.105269,2.014777,-1.494419
213,-2.173417,-1.165294,-0.835647,3.270208
218,-1.719211,-0.376136,-0.670348,3.603309
223,3.154140,0.561998,-0.968356,0.880286
234,-1.226230,2.325690,2.046296,-3.102184
280,-1.608206,0.385363,0.198772,3.073662
425,3.532026,-1.280305,1.397056,-1.040832
472,3.083159,-0.381372,-0.188685,-1.078924
476,3.447733,0.093765,-0.941333,-1.101084
555,-3.222834,-1.113132,-1.838212,1.790830


In [78]:
np.sign(data) * 3

,0,1,2,3
0,-3.0,-3.0,3.0,-3.0
1,3.0,-3.0,-3.0,3.0
2,3.0,3.0,3.0,-3.0
3,-3.0,3.0,-3.0,-3.0
4,-3.0,3.0,3.0,-3.0
...,...,...,...,...
995,-3.0,-3.0,-3.0,3.0
996,-3.0,-3.0,-3.0,-3.0
997,3.0,3.0,3.0,-3.0
998,-3.0,3.0,-3.0,-3.0


In [79]:
data[np.abs(data) > 3] = np.sign(data) * 3
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.029395,0.039221,0.022261,0.023561
std,1.024545,1.017294,0.987705,1.000819
min,-3.000000,-2.663346,-2.780534,-3.000000
25%,-0.744103,-0.634721,-0.655190,-0.639902
50%,-0.032687,-0.011789,0.015264,0.041739
75%,0.668105,0.756746,0.714148,0.697417
max,3.000000,3.000000,2.897205,3.000000


The statement np.sign(data) produces 1 and –1 values based on whether the values
in data are positive or negative:

In [80]:
np.sign(data).head()

,0,1,2,3
0,-1.0,-1.0,1.0,-1.0
1,1.0,-1.0,-1.0,1.0
2,1.0,1.0,1.0,-1.0
3,-1.0,1.0,-1.0,-1.0
4,-1.0,1.0,1.0,-1.0


### **Permutation and Random Sampling**

Permuting (randomly reordering) a Series or the rows in a DataFrame is easy to do
using the numpy.random.permutation function. Calling permutation with the length
of the axis you want to permute produces an array of integers indicating the new
ordering:

In [81]:
df = pd.DataFrame(np.arange(5 * 4).reshape((5, 4)))
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [82]:
sampler = np.random.permutation(4)
sampler

array([3, 1, 2, 0])

That array can then be used in iloc-based indexing or the equivalent take function:


In [83]:
df.take(sampler, axis=1)

,3,1,2,0
0,3,1,2,0
1,7,5,6,4
2,11,9,10,8
3,15,13,14,12
4,19,17,18,16


To select a random subset without replacement, you can use the sample method on
Series and DataFrame:

In [84]:
df.sample(n=6, axis=0, replace=True)

,0,1,2,3
1,4,5,6,7
2,8,9,10,11
4,16,17,18,19
2,8,9,10,11
4,16,17,18,19
3,12,13,14,15


To generate a sample with replacement (to allow repeat choices), pass replace=True
to sample:

In [85]:
choices = pd.Series([5, -7, -1, 6, 4])
choices

0    5
1   -7
2   -1
3    6
4    4
dtype: int64

In [86]:
choices.sample(n=10, replace=True)

2   -1
1   -7
3    6
3    6
3    6
4    4
1   -7
3    6
4    4
0    5
dtype: int64

### **Computing Indicator/Dummy Variables**

Another type of transformation for statistical modeling or machine learning applica‐
tions is converting a categorical variable into a “dummy” or “indicator” matrix. If a
column in a DataFrame has k distinct values, you would derive a matrix or Data‐
Frame with k columns containing all 1s and 0s. pandas has a get_dummies function
for doing this, though devising one yourself is not difficult. Let’s return to an earlier
example DataFrame:

In [87]:
df = pd.DataFrame({'key':['b', 'b', 'a', 'c', 'a', 'b'],
              'data1': range(6)
             })
df

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [88]:
pd.get_dummies(df['key'])

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In some cases, you may want to add a prefix to the columns in the indicator Data‐
Frame, which can then be merged with the other data. get_dummies has a prefix argu‐
ment for doing this:

In [89]:
dummies = pd.get_dummies(df['key'], prefix='key')
dummies

,key_a,key_b,key_c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [90]:
df_with_dummies = df[['data1']].join(dummies)
df_with_dummies

,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


If a row in a DataFrame belongs to multiple categories, things are a bit more compli‐
cated. Let’s look at the MovieLens 1M dataset, 

In [91]:
mnames = ['movie_id', 'title', 'genres']

In [92]:
movies = pd.read_table('./pydata-book/datasets/movielens/movies.dat', sep='::', engine='python', header=None, names=mnames)
movies[:10]

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


Adding indicator variables for each genre requires a little bit of wrangling. First, we
extract the list of unique genres in the dataset:

In [93]:
all_genres = []
for x in movies.genres:
    all_genres.extend(x.split('|'))
    # all_genres.append()


In [94]:
genres = pd.unique(all_genres)
genres

array(['Animation', "Children's", 'Comedy', 'Adventure', 'Fantasy',
       'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror',
       'Sci-Fi', 'Documentary', 'War', 'Musical', 'Mystery', 'Film-Noir',
       'Western'], dtype=object)

In [95]:
zero_matrix = np.zeros((len(movies), len(genres)))
zero_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [96]:
dummies = pd.DataFrame(zero_matrix, columns=genres)
dummies

,Animation,Children's,Comedy,Adventure,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Sci-Fi,Documentary,War,Musical,Mystery,Film-Noir,Western
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3878,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3879,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3880,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3881,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [97]:
gen = movies.genres[2]
gen

'Comedy|Romance'

In [98]:
gen.split('|')

['Comedy', 'Romance']

In [99]:
dummies.columns.get_indexer(gen.split('|'))

array([2, 5], dtype=int64)

In [100]:
movies.genres

0        Animation|Children's|Comedy
1       Adventure|Children's|Fantasy
2                     Comedy|Romance
3                       Comedy|Drama
4                             Comedy
                    ...             
3878                          Comedy
3879                           Drama
3880                           Drama
3881                           Drama
3882                  Drama|Thriller
Name: genres, Length: 3883, dtype: object

In [101]:
for i, genre in enumerate(movies.genres):
    indices = dummies.columns.get_indexer(genre.split('|'))
    dummies.iloc[i, indices] = 1


In [102]:
movies_windic = movies.join(dummies.add_prefix('Genre_'))
movies_windic.iloc[10]

movie_id                                         11
title                American President, The (1995)
genres                         Comedy|Drama|Romance
Genre_Animation                                 0.0
Genre_Children's                                0.0
Genre_Comedy                                    1.0
Genre_Adventure                                 0.0
Genre_Fantasy                                   0.0
Genre_Romance                                   1.0
Genre_Drama                                     1.0
Genre_Action                                    0.0
Genre_Crime                                     0.0
Genre_Thriller                                  0.0
Genre_Horror                                    0.0
Genre_Sci-Fi                                    0.0
Genre_Documentary                               0.0
Genre_War                                       0.0
Genre_Musical                                   0.0
Genre_Mystery                                   0.0
Genre_Film-N

A useful recipe for statistical applications is to combine get_dummies with a discreti‐
zation function like cut:

In [103]:
np.random.seed(12345)

In [104]:
values = np.random.rand(10)
values

array([0.92961609, 0.31637555, 0.18391881, 0.20456028, 0.56772503,
       0.5955447 , 0.96451452, 0.6531771 , 0.74890664, 0.65356987])

In [105]:
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]
bins

[0, 0.2, 0.4, 0.6, 0.8, 1]

In [106]:
pd.get_dummies(pd.cut(values, bins))

,"(0.0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1.0]"
0,0,0,0,0,1
1,0,1,0,0,0
2,1,0,0,0,0
3,0,1,0,0,0
4,0,0,1,0,0
5,0,0,1,0,0
6,0,0,0,0,1
7,0,0,0,1,0
8,0,0,0,1,0
9,0,0,0,1,0


## 7.3  String Manipulation

Python has long been a popular raw data manipulation language in part due to its
ease of use for string and text processing. Most text operations are made simple with
the string object’s built-in methods. For more complex pattern matching and text
manipulations, regular expressions may be needed. pandas adds to the mix by ena‐
bling you to apply string and regular expressions concisely on whole arrays of data,
additionally handling the annoyance of missing data.

### **String Object Methods**

In many string munging and scripting applications, built-in string methods are suffi‐
cient. As an example, a comma-separated string can be broken into pieces with
split:

In [112]:
val = 'a,b,   guido'
val.split(',')

['a', 'b', '   guido']

split is often combined with strip to trim whitespace (including line breaks):


In [116]:
pieces = [x.strip() for x in val.split(',')]
pieces

['a', 'b', 'guido']

In [117]:
first, second, third = pieces
first

'a'

These substrings could be concatenated together with a two-colon delimiter using
addition:

In [119]:
first + '::' + second + '::' + third

'a::b::guido'

But this isn’t a practical generic method. A faster and more Pythonic way is to pass a
list or tuple to the join method on the string '::':

In [120]:
'::'.join(pieces)

'a::b::guido'

Other methods are concerned with locating substrings. Using Python’s in keyword is
the best way to detect a substring, though index and find can also be used:

In [121]:
'guido' in val

True

In [124]:
val.index('guido'), val.find(':')

(7, -1)

Note the difference between find and index is that index raises an exception if the
string isn’t found (versus returning –1):

In [125]:
val.index(':')

ValueError: substring not found

In [127]:
val.count(' ')

3

replace will substitute occurrences of one pattern for another. It is commonly used
to delete patterns, too, by passing an empty string:

In [128]:
val.replace(',', '::')

'a::b::   guido'

In [129]:
val.replace(',', '')

'ab   guido'

In [131]:
val.replace(' ', '')

'a,b,guido'

### **Regular Expressions**

Regular expressions provide a flexible way to search or match (often more complex)
string patterns in text. A single expression, commonly called a regex, is a string
formed according to the regular expression language. Python’s built-in re module is
responsible for applying regular expressions to strings; I’ll give a number of examples
of its use here.

The re module functions fall into three categories: pattern matching, substitution,
and splitting. Naturally these are all related; a regex describes a pattern to locate in the
text, which can then be used for many purposes. Let’s look at a simple example:

suppose we wanted to split a string with a variable number of whitespace characters
(tabs, spaces, and newlines). The regex describing one or more whitespace characters
is \s+:

In [132]:
import re

In [136]:
text = "foo bar\t baz \tqux"
re.split('\s+', text, maxsplit=2)

['foo', 'bar', 'baz \tqux']

When you call re.split('\s+', text), the regular expression is first compiled, and
then its split method is called on the passed text. You can compile the regex yourself
with re.compile, forming a reusable regex object:

In [137]:
regex = re.compile('\s+')
regex

re.compile(r'\s+', re.UNICODE)

In [138]:
regex.split(text)

['foo', 'bar', 'baz', 'qux']

If, instead, you wanted to get a list of all patterns matching the regex, you can use the
findall method:

In [139]:
regex.findall(text)

[' ', '\t ', ' \t']

Creating a regex object with re.compile is highly recommended if you intend to
apply the same expression to many strings; doing so will save CPU cycles.

match and search are closely related to findall. While findall returns all matches
in a string, search returns only the first match. More rigidly, match only matches at
the beginning of the string. As a less trivial example, let’s consider a block of text and
a regular expression capable of identifying most email addresses:

In [140]:
text = """Dave dave@google.com
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com
"""
text

'Dave dave@google.com\nSteve steve@gmail.com\nRob rob@gmail.com\nRyan ryan@yahoo.com\n'

In [142]:
pattern = r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}'
pattern

'[A-Z0-9._%+-]+@[A-Z0-9.-]+\\.[A-Z]{2,4}'

In [144]:
regex = re.compile(pattern, flags=re.IGNORECASE)
regex

re.compile(r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}', re.IGNORECASE|re.UNICODE)

Using findall on the text produces a list of the email addresses:

In [145]:
re.findall(regex, text)

['dave@google.com', 'steve@gmail.com', 'rob@gmail.com', 'ryan@yahoo.com']

search returns a special match object for the first email address in the text. For the
preceding regex, the match object can only tell us the start and end position of the
pattern in the string:

In [148]:
m = re.search(regex, text)
m

<re.Match object; span=(5, 20), match='dave@google.com'>

In [150]:
text[m.start():m.end()]

'dave@google.com'

regex.match returns None, as it only will match if the pattern occurs at the start of the
string:

In [155]:
print(regex.match(text))

None


Relatedly, sub will return a new string with occurrences of the pattern replaced by the
a new string:

In [154]:
print(regex.sub('REDACTED', text))

Dave REDACTED
Steve REDACTED
Rob REDACTED
Ryan REDACTED



Suppose you wanted to find email addresses and simultaneously segment each
address into its three components: username, domain name, and domain suffix. To
do this, put parentheses around the parts of the pattern to segment:

In [156]:
pattern = r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})'
pattern

'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\\.([A-Z]{2,4})'

In [157]:
regex = re.compile(pattern, flags=re.IGNORECASE)

In [165]:
m = (regex.match('dave@google.com'))
m

<re.Match object; span=(0, 15), match='dave@google.com'>

In [169]:
m.groups()

('dave', 'google', 'com')

findall returns a list of tuples when the pattern has groups:


In [170]:
regex.findall(text)

[('dave', 'google', 'com'),
 ('steve', 'gmail', 'com'),
 ('rob', 'gmail', 'com'),
 ('ryan', 'yahoo', 'com')]

sub also has access to groups in each match using special symbols like \1 and \2. The
symbol \1 corresponds to the first matched group, \2 corresponds to the second, and
so forth:

In [171]:
print(regex.sub(r'Username: \1, Domain: \2, Suffix: \3', text))

Dave Username: dave, Domain: google, Suffix: com
Steve Username: steve, Domain: gmail, Suffix: com
Rob Username: rob, Domain: gmail, Suffix: com
Ryan Username: ryan, Domain: yahoo, Suffix: com



### **Vectorized String Functions in pandas**

Cleaning up a messy dataset for analysis often requires a lot of string munging and
regularization. To complicate matters, a column containing strings will sometimes
have missing data:

In [174]:
data = {'Dave': 'dave@google.com', 'Steve': 'steve@gmail.com',
 'Rob': 'rob@gmail.com', 'Wes': np.nan}
data

{'Dave': 'dave@google.com',
 'Steve': 'steve@gmail.com',
 'Rob': 'rob@gmail.com',
 'Wes': nan}

In [175]:
data = pd.Series(data)
data

Dave     dave@google.com
Steve    steve@gmail.com
Rob        rob@gmail.com
Wes                  NaN
dtype: object

In [177]:
data.isna()

Dave     False
Steve    False
Rob      False
Wes       True
dtype: bool

In [183]:
data.fillna(data.to_list()[0])

Dave     dave@google.com
Steve    steve@gmail.com
Rob        rob@gmail.com
Wes      dave@google.com
dtype: object

You can apply string and regular expression methods can be applied (passing a
lambda or other function) to each value using data.map, but it will fail on the NA
(null) values. To cope with this, Series has array-oriented methods for string opera‐
tions that skip NA values. These are accessed through Series’s str attribute; for exam‐
ple, we could check whether each email address has 'gmail' in it with str.contains:

In [186]:
data.str.contains('g')

Dave     True
Steve    True
Rob      True
Wes       NaN
dtype: object

Regular expressions can be used, too, along with any re options like IGNORECASE:


In [187]:
pattern

'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\\.([A-Z]{2,4})'

In [189]:
data.str.findall(pattern, flags=re.IGNORECASE)

Dave     [(dave, google, com)]
Steve    [(steve, gmail, com)]
Rob        [(rob, gmail, com)]
Wes                        NaN
dtype: object

There are a couple of ways to do vectorized element retrieval. Either use str.get or
index into the str attribute:

In [191]:
matches = data.str.match(pattern, flags=re.IGNORECASE)
matches

Dave     True
Steve    True
Rob      True
Wes       NaN
dtype: object

In [195]:
matches.get(3)

nan

## 7.4 Conclusion

Effective data preparation can significantly improve productive by enabling you to
spend more time analyzing data and less time getting it ready for analysis. We have
explored a number of tools in this chapter, but the coverage here is by no means com‐
prehensive. In the next chapter, we will explore pandas’s joining and grouping func‐
tionality.